In [ ]:
import numpy as np
import pandas as pd

class simpleOutlierMaskSystem:
    def __init__(self, df: pd.DataFrame, column: str):
        self.df = df
        self.column = column
        self.mask = None

    def method(self, tool_name: str, config: dict):
        if tool_name == "Z-Score":
            self.mask = self._z_score(config)

        elif tool_name == "Percentile":
            self.mask = self._percentile(config)

        elif tool_name == "Quantile":
            self.mask = self._quantile(config)

        elif tool_name == "IQR":
            self.mask = self._iqr(config)

        else:
            return "Invalid detection method"

        return self.mask

    def _z_score(self, config: dict):
        threshold = config.get("threshold")
        if threshold is None:
            return "threshold required"

        x = self.df[self.column]
        mean = x.mean()
        std = x.std()

        z = (x - mean) / std
        mask = np.abs(z) > threshold

        return mask

    def _percentile(self, config: dict):
        lower = config.get("lower")
        upper = config.get("upper")

        if lower is None or upper is None:
            return "lower and upper required"

        x = self.df[self.column]
        lower_val = np.percentile(x, lower)
        upper_val = np.percentile(x, upper)

        mask = (x < lower_val) | (x > upper_val)

        return mask

    def _quantile(self, config: dict):
        lower_q = config.get("lower_q")
        upper_q = config.get("upper_q")

        if lower_q is None or upper_q is None:
            return "lower_q and upper_q required"

        x = self.df[self.column]
        lower_val = x.quantile(lower_q)
        upper_val = x.quantile(upper_q)

        mask = (x < lower_val) | (x > upper_val)

        return mask

    def _iqr(self, config: dict):
        multiplier = config.get("multiplier")
        if multiplier is None:
            return "multiplier required"

        x = self.df[self.column]
        q1 = x.quantile(0.25)
        q3 = x.quantile(0.75)
        iqr = q3 - q1

        lower_bound = q1 - multiplier * iqr
        upper_bound = q3 + multiplier * iqr

        mask = (x < lower_bound) | (x > upper_bound)

        return mask